In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import img_to_array
from loader_util.preprocessing import AspectAwarePreprocessor, ImageToArrayPreprocessor
from loader_util.datasets import SimpleDatasetLoader
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from loader_util.nn.conv import LeNet
from imutils import paths
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes._axes as axes
import cv2
import os
sns.set()

In [3]:
dataFolder = r"C:\Users\mhasa\Google Drive\Tutorial " \
             r"Corner\PYTH\DeepLearning\DeepLearning-DL4CV\ImageDatasets\cellImages2"

args = {
    'dataset': dataFolder
}

imagePaths = list(paths.list_images(args['dataset']))

In [4]:
# initialise the preprocessors
sp = AspectAwarePreprocessor(64, 64)
iap = ImageToArrayPreprocessor()

# load the dataset
sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
data, labels = sdl.load(imagePaths, verbose=500)
data = data.astype('float') / 255.0

[INFO] processed 500/1337
[INFO] processed 1000/1337


In [5]:
le = LabelEncoder().fit(labels)
labels = to_categorical(le.transform(labels), 3)
trainx, testx, trainy, testy = train_test_split(data, labels, test_size=0.2,
                                                random_state=42, stratify=labels)

# imbalance
classTotals = labels.sum(axis=0)
classWeight = classTotals.max() / classTotals

In [6]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, vertical_flip=True, fill_mode="nearest")

In [7]:
from loader_util.nn.conv.alexnet import AlexNet

In [8]:
cpt = ModelCheckpoint('best_model_weights.hdf5', monitor="val_accuracy", 
                      save_best_only=True,
                     verbose=1, mode='max')
callbacks = [cpt]

In [9]:
epoch_no = 2000
opt = SGD(lr=0.005)
model = LeNet.build(width=64, height=64, depth=3, classes=3)
model.compile(loss='categorical_crossentropy', optimizer=opt, 
              metrics=['accuracy'])
H = model.fit_generator(aug.flow(trainx, trainy, batch_size=32 ),
  validation_data=(testx, testy), steps_per_epoch=len (trainx) // 32 ,
  epochs=epoch_no , verbose=1, callbacks=callbacks)

# H = model.fit(trainx,
#               trainy, validation_data=[testx, testy],
#               batch_size=32,
#               epochs=epoch_no,
#               verbose=1,
#               class_weight=classWeight)

Epoch 1/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0978 - accuracy: 0.3463
Epoch 00001: val_accuracy improved from -inf to 0.36194, saving model to best_model_weights.hdf5
33/33 [==============================] - 13s 397ms/step - loss: 1.0974 - accuracy: 0.3491 - val_loss: 1.0929 - val_accuracy: 0.3619
Epoch 2/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0958 - accuracy: 0.3532
Epoch 00002: val_accuracy did not improve from 0.36194
33/33 [==============================] - 2s 70ms/step - loss: 1.0955 - accuracy: 0.3558 - val_loss: 1.0905 - val_accuracy: 0.3433
Epoch 3/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0954 - accuracy: 0.3572
Epoch 00003: val_accuracy did not improve from 0.36194
33/33 [==============================] - 2s 64ms/step - loss: 1.0954 - accuracy: 0.3529 - val_loss: 1.0891 - val_accuracy: 0.3619
Epoch 4/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0903 - accuracy: 0.3623
Epoch 00004: val_

Epoch 29/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0631 - accuracy: 0.4677
Epoch 00029: val_accuracy did not improve from 0.60821
33/33 [==============================] - 2s 60ms/step - loss: 1.0642 - accuracy: 0.4667 - val_loss: 1.1031 - val_accuracy: 0.3619
Epoch 30/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0663 - accuracy: 0.4139
Epoch 00030: val_accuracy did not improve from 0.60821
33/33 [==============================] - 2s 70ms/step - loss: 1.0653 - accuracy: 0.4214 - val_loss: 1.0501 - val_accuracy: 0.3619
Epoch 31/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0701 - accuracy: 0.3770
Epoch 00031: val_accuracy did not improve from 0.60821
33/33 [==============================] - 2s 63ms/step - loss: 1.0714 - accuracy: 0.3703 - val_loss: 1.1253 - val_accuracy: 0.3619
Epoch 32/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0829 - accuracy: 0.3731
Epoch 00032: val_accuracy did not improve from 0.60821
33

Epoch 58/2000
31/33 [===========================>..] - ETA: 0s - loss: 1.0724 - accuracy: 0.4264
Epoch 00058: val_accuracy did not improve from 0.69403
33/33 [==============================] - 2s 57ms/step - loss: 1.0720 - accuracy: 0.4214 - val_loss: 1.0674 - val_accuracy: 0.6231
Epoch 59/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0611 - accuracy: 0.4567
Epoch 00059: val_accuracy did not improve from 0.69403
33/33 [==============================] - 2s 70ms/step - loss: 1.0607 - accuracy: 0.4542 - val_loss: 1.0240 - val_accuracy: 0.3657
Epoch 60/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0362 - accuracy: 0.4716
Epoch 00060: val_accuracy did not improve from 0.69403
33/33 [==============================] - 2s 67ms/step - loss: 1.0359 - accuracy: 0.4754 - val_loss: 1.0433 - val_accuracy: 0.3619
Epoch 61/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0572 - accuracy: 0.4229
Epoch 00061: val_accuracy did not improve from 0.69403
33

Epoch 87/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0358 - accuracy: 0.4448
Epoch 00087: val_accuracy did not improve from 0.78358
33/33 [==============================] - 2s 66ms/step - loss: 1.0363 - accuracy: 0.4417 - val_loss: 1.0082 - val_accuracy: 0.7164
Epoch 88/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0205 - accuracy: 0.5025
Epoch 00088: val_accuracy did not improve from 0.78358
33/33 [==============================] - 2s 75ms/step - loss: 1.0195 - accuracy: 0.5024 - val_loss: 1.0287 - val_accuracy: 0.3657
Epoch 89/2000
31/33 [===========================>..] - ETA: 0s - loss: 1.0210 - accuracy: 0.4546
Epoch 00089: val_accuracy did not improve from 0.78358
33/33 [==============================] - 2s 62ms/step - loss: 1.0179 - accuracy: 0.4590 - val_loss: 1.0029 - val_accuracy: 0.3619
Epoch 90/2000
32/33 [============================>.] - ETA: 0s - loss: 1.0170 - accuracy: 0.4531
Epoch 00090: val_accuracy did not improve from 0.78358
33

Epoch 116/2000
32/33 [============================>.] - ETA: 0s - loss: 0.9237 - accuracy: 0.5672
Epoch 00116: val_accuracy did not improve from 0.78358
33/33 [==============================] - 2s 62ms/step - loss: 0.9245 - accuracy: 0.5670 - val_loss: 0.9582 - val_accuracy: 0.3657
Epoch 117/2000
32/33 [============================>.] - ETA: 0s - loss: 0.9153 - accuracy: 0.4990
Epoch 00117: val_accuracy did not improve from 0.78358
33/33 [==============================] - 2s 56ms/step - loss: 0.9143 - accuracy: 0.5079 - val_loss: 0.8524 - val_accuracy: 0.7090
Epoch 118/2000
32/33 [============================>.] - ETA: 0s - loss: 0.9104 - accuracy: 0.5632
Epoch 00118: val_accuracy did not improve from 0.78358
33/33 [==============================] - 2s 61ms/step - loss: 0.9087 - accuracy: 0.5603 - val_loss: 0.9295 - val_accuracy: 0.4888
Epoch 119/2000
32/33 [============================>.] - ETA: 0s - loss: 0.9366 - accuracy: 0.5312
Epoch 00119: val_accuracy did not improve from 0.7835

...

Epoch 348/2000
32/33 [============================>.] - ETA: 0s - loss: 0.6413 - accuracy: 0.7313
Epoch 00348: val_accuracy did not improve from 0.79851
33/33 [==============================] - 2s 64ms/step - loss: 0.6423 - accuracy: 0.7281 - val_loss: 0.5634 - val_accuracy: 0.7463
Epoch 349/2000
31/33 [===========================>..] - ETA: 0s - loss: 0.5548 - accuracy: 0.7725
Epoch 00349: val_accuracy did not improve from 0.79851
33/33 [==============================] - 2s 61ms/step - loss: 0.5539 - accuracy: 0.7760 - val_loss: 0.6052 - val_accuracy: 0.7276
Epoch 350/2000
32/33 [============================>.] - ETA: 0s - loss: 0.4641 - accuracy: 0.8030
Epoch 00350: val_accuracy did not improve from 0.79851
33/33 [==============================] - 2s 65ms/step - loss: 0.4657 - accuracy: 0.8023 - val_loss: 0.4626 - val_accuracy: 0.7910
Epoch 351/2000
32/33 [============================>.] - ETA: 0s - loss: 0.4384 - accuracy: 0.7970
Epoch 00351: val_accuracy did not improve from 0.7985

Epoch 377/2000
32/33 [============================>.] - ETA: 0s - loss: 0.4756 - accuracy: 0.7721
Epoch 00377: val_accuracy did not improve from 0.82090
33/33 [==============================] - 2s 60ms/step - loss: 0.4710 - accuracy: 0.7734 - val_loss: 0.4381 - val_accuracy: 0.8134
Epoch 378/2000
32/33 [============================>.] - ETA: 0s - loss: 0.4014 - accuracy: 0.8338
Epoch 00378: val_accuracy did not improve from 0.82090
33/33 [==============================] - 2s 60ms/step - loss: 0.4001 - accuracy: 0.8341 - val_loss: 0.6278 - val_accuracy: 0.6716
Epoch 379/2000
32/33 [============================>.] - ETA: 0s - loss: 0.4153 - accuracy: 0.8279
Epoch 00379: val_accuracy did not improve from 0.82090
33/33 [==============================] - 2s 60ms/step - loss: 0.4129 - accuracy: 0.8303 - val_loss: 0.5450 - val_accuracy: 0.7351
Epoch 380/2000
32/33 [============================>.] - ETA: 0s - loss: 0.5720 - accuracy: 0.7463
Epoch 00380: val_accuracy did not improve from 0.8209

Epoch 406/2000
32/33 [============================>.] - ETA: 0s - loss: 0.4573 - accuracy: 0.7900
Epoch 00406: val_accuracy did not improve from 0.84701
33/33 [==============================] - 2s 58ms/step - loss: 0.4577 - accuracy: 0.7898 - val_loss: 0.4042 - val_accuracy: 0.8358
Epoch 407/2000
32/33 [============================>.] - ETA: 0s - loss: 0.6717 - accuracy: 0.7095
Epoch 00407: val_accuracy did not improve from 0.84701
33/33 [==============================] - 2s 61ms/step - loss: 0.6660 - accuracy: 0.7078 - val_loss: 0.4170 - val_accuracy: 0.8246
Epoch 408/2000
32/33 [============================>.] - ETA: 0s - loss: 0.3837 - accuracy: 0.8269
Epoch 00408: val_accuracy did not improve from 0.84701
33/33 [==============================] - 2s 60ms/step - loss: 0.3847 - accuracy: 0.8264 - val_loss: 0.5049 - val_accuracy: 0.7276
Epoch 409/2000
32/33 [============================>.] - ETA: 0s - loss: 0.4371 - accuracy: 0.8100
Epoch 00409: val_accuracy did not improve from 0.8470

Epoch 435/2000
32/33 [============================>.] - ETA: 0s - loss: 0.3609 - accuracy: 0.8448
Epoch 00435: val_accuracy did not improve from 0.85821
33/33 [==============================] - 2s 62ms/step - loss: 0.3567 - accuracy: 0.8476 - val_loss: 0.3801 - val_accuracy: 0.8172
Epoch 436/2000
32/33 [============================>.] - ETA: 0s - loss: 0.3363 - accuracy: 0.8587
Epoch 00436: val_accuracy did not improve from 0.85821
33/33 [==============================] - 2s 60ms/step - loss: 0.3340 - accuracy: 0.8602 - val_loss: 0.5685 - val_accuracy: 0.7052
Epoch 437/2000
32/33 [============================>.] - ETA: 0s - loss: 0.4820 - accuracy: 0.7871
Epoch 00437: val_accuracy did not improve from 0.85821
33/33 [==============================] - 2s 62ms/step - loss: 0.4767 - accuracy: 0.7898 - val_loss: 0.4166 - val_accuracy: 0.8284
Epoch 438/2000
32/33 [============================>.] - ETA: 0s - loss: 0.3084 - accuracy: 0.8896
Epoch 00438: val_accuracy did not improve from 0.8582

OSError: Unable to create file (unable to open file: name = 'best_model_weights.hdf5', errno = 22, error message = 'Invalid argument', flags = 13, o_flags = 302)

In [ ]:
# evaluate the network
preds = model.predict(testx, batch_size=32)
rep = classification_report(testy.argmax(axis=1), preds.argmax(axis=1), 
                            target_names=le.classes_)
print(rep)

In [ ]:
epochs = range(1,epoch_no+1)
loss = H.history['loss']
accuracy = H.history['acc']
val_loss = H.history['val_loss']
val_accuracy = H.history['val_acc']
plot_df = pd.DataFrame(data=np.c_[epochs,loss,accuracy,val_loss,val_accuracy], 
                       columns=['epochs','loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
sns.set(font_scale=1)
f, ax = plt.subplots(1, 1, figsize=(18,10))
sns.lineplot(data=plot_df, x='epochs', y='loss', ax=ax, label='train loss', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='accuracy', ax=ax, label='train accuracy', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='val_loss', ax=ax, label='val loss', linewidth=3)
sns.lineplot(data=plot_df, x='epochs', y='val_accuracy', ax=ax, label='val_accuracy', linewidth=3)
ax.set_ylabel('Loss or Accuracy')
ax.set_xlabel('Epochs')
plt.setp(ax.get_legend().get_texts(), fontsize='18'); # for legend text

In [ ]:
model.save('./checkpoint')